# Overview
1. Add CVE Descriptions to Top 25 removing newlines, tabs,... and escaping as required
2. Remove entries where CWE is not a CWE e.g. NVD-CWE-Insufficient-Info, UNSURE

In [34]:
import pandas as pd
import csv
import re





In [35]:
df_cve = pd.read_csv('./data_out/CVSSData.csv.gz', quoting=csv.QUOTE_ALL, escapechar='\\', compression='gzip')
df_cve

,CVE,Published,Description,AttackVector CVSS3,AttackComplexity CVSS3,PrivilegesRequired CVSS3,UserInteraction CVSS3,Scope CVSS3,ConfidentialityImpact CVSS3,IntegrityImpact CVSS3,...,AccessComplexity CVSS2,Authentication CVSS2,ConfidentialityImpact CVSS2,IntegrityImpact CVSS2,AvailabilityImpact CVSS2,BaseScore CVSS2,BaseSeverity CVSS2,ExploitabilityScore CVSS2,ImpactScore CVSS2,CWEs
0,CVE-2021-3002,2021-01-01,Seo Panel 4.8.0 allows reflected XSS via the s...,NETWORK,LOW,NONE,REQUIRED,CHANGED,LOW,LOW,...,MEDIUM,NONE,NONE,PARTIAL,NONE,4.3,Missing_Data,8.6,2.9,['CWE-79']
1,CVE-2021-3005,2021-01-03,MK-AUTH through 19.01 K4.9 allows remote attac...,NETWORK,LOW,LOW,NONE,UNCHANGED,LOW,NONE,...,LOW,SINGLE,PARTIAL,NONE,NONE,4.0,Missing_Data,8.0,2.9,['NVD-CWE-noinfo']
2,CVE-2021-3004,2021-01-03,The _deposit function in the smart contract im...,NETWORK,LOW,NONE,NONE,UNCHANGED,NONE,HIGH,...,LOW,NONE,NONE,PARTIAL,NONE,5.0,Missing_Data,10.0,2.9,['CWE-682']
3,CVE-2021-3006,2021-01-03,The breed function in the smart contract imple...,NETWORK,LOW,NONE,NONE,UNCHANGED,NONE,HIGH,...,LOW,NONE,NONE,PARTIAL,NONE,5.0,Missing_Data,10.0,2.9,['NVD-CWE-Other']
4,CVE-2021-3007,2021-01-04,"Laminas Project laminas-http before 2.14.2, an...",NETWORK,LOW,NONE,NONE,UNCHANGED,HIGH,HIGH,...,LOW,NONE,PARTIAL,PARTIAL,PARTIAL,7.5,Missing_Data,10.0,6.4,['CWE-502']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46311,CVE-2022-48942,2024-08-22,"In the Linux kernel, the following vulnerabili...",LOCAL,LOW,LOW,NONE,UNCHANGED,NONE,NONE,...,Missing_Data,Missing_Data,Missing_Data,Missing_Data,Missing_Data,0.0,Missing_Data,0.0,0.0,['CWE-476']
46312,CVE-2022-48943,2024-08-22,"In the Linux kernel, the following vulnerabili...",LOCAL,LOW,LOW,NONE,UNCHANGED,HIGH,HIGH,...,Missing_Data,Missing_Data,Missing_Data,Missing_Data,Missing_Data,0.0,Missing_Data,0.0,0.0,['NVD-CWE-noinfo']
46313,CVE-2022-48926,2024-08-22,"In the Linux kernel, the following vulnerabili...",LOCAL,LOW,LOW,NONE,UNCHANGED,HIGH,HIGH,...,Missing_Data,Missing_Data,Missing_Data,Missing_Data,Missing_Data,0.0,Missing_Data,0.0,0.0,['NVD-CWE-noinfo']
46314,CVE-2022-48936,2024-08-22,"In the Linux kernel, the following vulnerabili...",LOCAL,LOW,LOW,NONE,UNCHANGED,NONE,NONE,...,Missing_Data,Missing_Data,Missing_Data,Missing_Data,Missing_Data,0.0,Missing_Data,0.0,0.0,['NVD-CWE-noinfo']


## Add CVE Description to CWE Observed and Top25 so they have the same format
"CVE","Description","Chains","Weakness_Description"

In [36]:
df_observed = pd.read_csv('data_out/observed_examples.csv')

df_observed

,CWE-ID,CVE-ID,Description
0,CWE-1004,CVE-2022-24045,Web application for a room automation system h...
1,CWE-1004,CVE-2014-3852,CMS written in Python does not include the HTT...
2,CWE-1004,CVE-2015-4138,Appliance for managing encrypted communication...
3,CWE-1007,CVE-2013-7236,web forum allows impersonation of users with h...
4,CWE-1007,CVE-2012-0584,Improper character restriction in URLs in web ...
...,...,...,...
2977,CWE-98,CVE-2004-0127,Directory traversal vulnerability in PHP inclu...
2978,CWE-98,CVE-2005-1971,Directory traversal vulnerability in PHP inclu...
2979,CWE-98,CVE-2005-3335,"PHP file inclusion issue, both remote and loca..."
2980,CWE-98,CVE-2009-1936,chain: library file sends a redirect if it is ...


In [37]:
df_observed.rename(columns={'CWE-ID': 'CWE', 'CVE-ID': 'CVE', 'Description': 'Weakness_Description'}, inplace=True)
df_observed

,CWE,CVE,Weakness_Description
0,CWE-1004,CVE-2022-24045,Web application for a room automation system h...
1,CWE-1004,CVE-2014-3852,CMS written in Python does not include the HTT...
2,CWE-1004,CVE-2015-4138,Appliance for managing encrypted communication...
3,CWE-1007,CVE-2013-7236,web forum allows impersonation of users with h...
4,CWE-1007,CVE-2012-0584,Improper character restriction in URLs in web ...
...,...,...,...
2977,CWE-98,CVE-2004-0127,Directory traversal vulnerability in PHP inclu...
2978,CWE-98,CVE-2005-1971,Directory traversal vulnerability in PHP inclu...
2979,CWE-98,CVE-2005-3335,"PHP file inclusion issue, both remote and loca..."
2980,CWE-98,CVE-2009-1936,chain: library file sends a redirect if it is ...


In [38]:
df_observed = df_observed.merge(df_cve[['CVE', 'Description']], on='CVE', how='left')
df_observed

,CWE,CVE,Weakness_Description,Description
0,CWE-1004,CVE-2022-24045,Web application for a room automation system h...,A vulnerability has been identified in Desigo ...
1,CWE-1004,CVE-2014-3852,CMS written in Python does not include the HTT...,NaN
2,CWE-1004,CVE-2015-4138,Appliance for managing encrypted communication...,NaN
3,CWE-1007,CVE-2013-7236,web forum allows impersonation of users with h...,NaN
4,CWE-1007,CVE-2012-0584,Improper character restriction in URLs in web ...,NaN
...,...,...,...,...
2977,CWE-98,CVE-2004-0127,Directory traversal vulnerability in PHP inclu...,NaN
2978,CWE-98,CVE-2005-1971,Directory traversal vulnerability in PHP inclu...,NaN
2979,CWE-98,CVE-2005-3335,"PHP file inclusion issue, both remote and loca...",NaN
2980,CWE-98,CVE-2009-1936,chain: library file sends a redirect if it is ...,NaN


In [39]:
# Reorder columns
df_observed = df_observed[['CVE', 'CWE', 'Description', 'Weakness_Description']]
df_observed

,CVE,CWE,Description,Weakness_Description
0,CVE-2022-24045,CWE-1004,A vulnerability has been identified in Desigo ...,Web application for a room automation system h...
1,CVE-2014-3852,CWE-1004,NaN,CMS written in Python does not include the HTT...
2,CVE-2015-4138,CWE-1004,NaN,Appliance for managing encrypted communication...
3,CVE-2013-7236,CWE-1007,NaN,web forum allows impersonation of users with h...
4,CVE-2012-0584,CWE-1007,NaN,Improper character restriction in URLs in web ...
...,...,...,...,...
2977,CVE-2004-0127,CWE-98,NaN,Directory traversal vulnerability in PHP inclu...
2978,CVE-2005-1971,CWE-98,NaN,Directory traversal vulnerability in PHP inclu...
2979,CVE-2005-3335,CWE-98,NaN,"PHP file inclusion issue, both remote and loca..."
2980,CVE-2009-1936,CWE-98,NaN,chain: library file sends a redirect if it is ...


In [40]:
df_observed.to_csv("./data_out/observed_with_cve_descriptions.csv", index=False, quoting=csv.QUOTE_ALL, escapechar='\\')

## Top25

In [41]:
df = pd.read_csv('./data_in/top25-mitre-mapping-analysis-2023-public.csv')

df

,CVE,New CWE,Reasoning
0,CVE-2021-0674,CWE-20,"(Chains: CWE-20->CWE-125) Desc: ""out of bounds..."
1,CVE-2021-0674,CWE-125,"(Chains: CWE-20->CWE-125) Desc: ""out of bounds..."
2,CVE-2021-0676,CWE-20,"(Chains: CWE-20->CWE-125) Desc: ""out of bounds..."
3,CVE-2021-0676,CWE-125,"(Chains: CWE-20->CWE-125) Desc: ""out of bounds..."
4,CVE-2021-0677,CWE-190,"(Chains: CWE-190->CWE-125) Desc: ""out of bound..."
...,...,...,...
9707,CVE-2022-29897,CWE-20,"""due to an improper input validation"". ref men..."
9708,CVE-2022-29922,CWE-20,"""Improper Input Validation"" in desc. NVD vendo..."
9709,CVE-2022-3001,CWE-20,"""improper input handling"" and NVD Reference ma..."
9710,CVE-2022-30232,CWE-20,"desc: ""CWE-20: Improper Input Validation"" and ..."


In [42]:
df = df.merge(df_cve[['CVE', 'Description']], on='CVE', how='left')
df

,CVE,New CWE,Reasoning,Description
0,CVE-2021-0674,CWE-20,"(Chains: CWE-20->CWE-125) Desc: ""out of bounds...","In alac decoder, there is a possible out of bo..."
1,CVE-2021-0674,CWE-125,"(Chains: CWE-20->CWE-125) Desc: ""out of bounds...","In alac decoder, there is a possible out of bo..."
2,CVE-2021-0676,CWE-20,"(Chains: CWE-20->CWE-125) Desc: ""out of bounds...","In geniezone driver, there is a possible out o..."
3,CVE-2021-0676,CWE-125,"(Chains: CWE-20->CWE-125) Desc: ""out of bounds...","In geniezone driver, there is a possible out o..."
4,CVE-2021-0677,CWE-190,"(Chains: CWE-190->CWE-125) Desc: ""out of bound...","In ccu driver, there is a possible out of boun..."
...,...,...,...,...
9707,CVE-2022-29897,CWE-20,"""due to an improper input validation"". ref men...",On various RAD-ISM-900-EN-* devices by PHOENIX...
9708,CVE-2022-29922,CWE-20,"""Improper Input Validation"" in desc. NVD vendo...",Improper Input Validation vulnerability in the...
9709,CVE-2022-3001,CWE-20,"""improper input handling"" and NVD Reference ma...",This vulnerability exists in Milesight Video M...
9710,CVE-2022-30232,CWE-20,"desc: ""CWE-20: Improper Input Validation"" and ...",A CWE-20: Improper Input Validation vulnerabil...


## Check for CVE Descriptions that are null

* 1 row has null description - CVE-2021-44228 - this is because the CVE was rejected.
* But we can still add the description from the CVE https://nvd.nist.gov/vuln/detail/CVE-2021-42248#VulnChangeHistorySection
* "GJSON <= 1.9.2 allows attackers to cause a redos via crafted JSON input."


In [43]:
df['Description'].isnull().sum()

np.int64(1)

In [44]:
null_description_rows = df[df['Description'].isnull()]
null_description_rows

,CVE,New CWE,Reasoning,Description
2849,CVE-2021-42248,CWE-1333,"""redos"" in desc",NaN


In [45]:
df.loc[2849, 'Description'] = "GJSON <= 1.9.2 allows attackers to cause a redos via crafted JSON input."
df['Description'].isnull().sum()

np.int64(0)

In [46]:

# The regular expression r'CWE-\d+$' breaks down as follows:

# CWE-: Matches the literal string "CWE-"
# \d+: Matches one or more digits
# $: Ensures that the digits are at the end of the string

df = df[df['New CWE'].str.match(r'CWE-\d+$')]
df

,CVE,New CWE,Reasoning,Description
0,CVE-2021-0674,CWE-20,"(Chains: CWE-20->CWE-125) Desc: ""out of bounds...","In alac decoder, there is a possible out of bo..."
1,CVE-2021-0674,CWE-125,"(Chains: CWE-20->CWE-125) Desc: ""out of bounds...","In alac decoder, there is a possible out of bo..."
2,CVE-2021-0676,CWE-20,"(Chains: CWE-20->CWE-125) Desc: ""out of bounds...","In geniezone driver, there is a possible out o..."
3,CVE-2021-0676,CWE-125,"(Chains: CWE-20->CWE-125) Desc: ""out of bounds...","In geniezone driver, there is a possible out o..."
4,CVE-2021-0677,CWE-190,"(Chains: CWE-190->CWE-125) Desc: ""out of bound...","In ccu driver, there is a possible out of boun..."
...,...,...,...,...
9707,CVE-2022-29897,CWE-20,"""due to an improper input validation"". ref men...",On various RAD-ISM-900-EN-* devices by PHOENIX...
9708,CVE-2022-29922,CWE-20,"""Improper Input Validation"" in desc. NVD vendo...",Improper Input Validation vulnerability in the...
9709,CVE-2022-3001,CWE-20,"""improper input handling"" and NVD Reference ma...",This vulnerability exists in Milesight Video M...
9710,CVE-2022-30232,CWE-20,"desc: ""CWE-20: Improper Input Validation"" and ...",A CWE-20: Improper Input Validation vulnerabil...


In [47]:
#df = df[df['New CWE'].str.startswith('CWE-')]
#df

In [48]:
# replace any sequence of whitespace characters (including newlines, tabs, and multiple spaces) with a single space.
df['Description'] = df['Description'].str.replace('\s+', ' ', regex=True)


<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_46603/1618951606.py:2: SyntaxWarning: invalid escape sequence '\s'
  df['Description'] = df['Description'].str.replace('\s+', ' ', regex=True)
/tmp/ipykernel_46603/1618951606.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Description'] = df['Description'].str.replace('\s+', ' ', regex=True)


## Split the Reasoning column into Chains and Weakness_Description


In [49]:

def extract_info(reasoning):
    # Extract Chains
    chains_match = re.search(r'\(Chains: (.*?)\)', reasoning)
    chains = chains_match.group(1) if chains_match else ''

    # Extract Weakness Description
    desc_match = re.search(r'Desc: "(.*?)"', reasoning)
    if not desc_match:
        desc_match = re.search(r'"(.*?)"', reasoning)
    weakness_description = desc_match.group(1) if desc_match else ''

    return pd.Series({'Chains': chains, 'Weakness_Description': weakness_description})

In [50]:
# Apply the extraction function to the 'Reasoning' column
new_columns = df['Reasoning'].apply(extract_info)

# Add the new columns to the DataFrame
df = pd.concat([df, new_columns], axis=1)

df

,CVE,New CWE,Reasoning,Description,Chains,Weakness_Description
0,CVE-2021-0674,CWE-20,"(Chains: CWE-20->CWE-125) Desc: ""out of bounds...","In alac decoder, there is a possible out of bo...",CWE-20->CWE-125,out of bounds read due to an incorrect bounds ...
1,CVE-2021-0674,CWE-125,"(Chains: CWE-20->CWE-125) Desc: ""out of bounds...","In alac decoder, there is a possible out of bo...",CWE-20->CWE-125,out of bounds read due to an incorrect bounds ...
2,CVE-2021-0676,CWE-20,"(Chains: CWE-20->CWE-125) Desc: ""out of bounds...","In geniezone driver, there is a possible out o...",CWE-20->CWE-125,out of bounds read due to an incorrect bounds ...
3,CVE-2021-0676,CWE-125,"(Chains: CWE-20->CWE-125) Desc: ""out of bounds...","In geniezone driver, there is a possible out o...",CWE-20->CWE-125,out of bounds read due to an incorrect bounds ...
4,CVE-2021-0677,CWE-190,"(Chains: CWE-190->CWE-125) Desc: ""out of bound...","In ccu driver, there is a possible out of boun...",CWE-190->CWE-125,out of bounds read due to an integer overflow
...,...,...,...,...,...,...
9707,CVE-2022-29897,CWE-20,"""due to an improper input validation"". ref men...",On various RAD-ISM-900-EN-* devices by PHOENIX...,,due to an improper input validation
9708,CVE-2022-29922,CWE-20,"""Improper Input Validation"" in desc. NVD vendo...",Improper Input Validation vulnerability in the...,,Improper Input Validation
9709,CVE-2022-3001,CWE-20,"""improper input handling"" and NVD Reference ma...",This vulnerability exists in Milesight Video M...,,improper input handling
9710,CVE-2022-30232,CWE-20,"desc: ""CWE-20: Improper Input Validation"" and ...",A CWE-20: Improper Input Validation vulnerabil...,,CWE-20: Improper Input Validation


In [51]:
df.to_csv("./data_out/top25-mitre-mapping-analysis-2023-public_with_cve_descriptions_with_reasoning.csv", index=False, quoting=csv.QUOTE_ALL, escapechar='\\')

In [52]:
df.drop('Reasoning', axis=1, inplace=True)
df.rename(columns={'New CWE': 'CWE'}, inplace=True)
df


,CVE,CWE,Description,Chains,Weakness_Description
0,CVE-2021-0674,CWE-20,"In alac decoder, there is a possible out of bo...",CWE-20->CWE-125,out of bounds read due to an incorrect bounds ...
1,CVE-2021-0674,CWE-125,"In alac decoder, there is a possible out of bo...",CWE-20->CWE-125,out of bounds read due to an incorrect bounds ...
2,CVE-2021-0676,CWE-20,"In geniezone driver, there is a possible out o...",CWE-20->CWE-125,out of bounds read due to an incorrect bounds ...
3,CVE-2021-0676,CWE-125,"In geniezone driver, there is a possible out o...",CWE-20->CWE-125,out of bounds read due to an incorrect bounds ...
4,CVE-2021-0677,CWE-190,"In ccu driver, there is a possible out of boun...",CWE-190->CWE-125,out of bounds read due to an integer overflow
...,...,...,...,...,...
9707,CVE-2022-29897,CWE-20,On various RAD-ISM-900-EN-* devices by PHOENIX...,,due to an improper input validation
9708,CVE-2022-29922,CWE-20,Improper Input Validation vulnerability in the...,,Improper Input Validation
9709,CVE-2022-3001,CWE-20,This vulnerability exists in Milesight Video M...,,improper input handling
9710,CVE-2022-30232,CWE-20,A CWE-20: Improper Input Validation vulnerabil...,,CWE-20: Improper Input Validation


In [53]:
df = df.drop_duplicates(subset=['CVE', 'Description', 'Chains'])
#df = df.reset_index(drop=True)
df.to_csv("./data_out/top25-mitre-mapping-analysis-2023-public_with_cve_descriptions.csv", index=False, quoting=csv.QUOTE_ALL, escapechar='\\')
df

,CVE,CWE,Description,Chains,Weakness_Description
0,CVE-2021-0674,CWE-20,"In alac decoder, there is a possible out of bo...",CWE-20->CWE-125,out of bounds read due to an incorrect bounds ...
2,CVE-2021-0676,CWE-20,"In geniezone driver, there is a possible out o...",CWE-20->CWE-125,out of bounds read due to an incorrect bounds ...
4,CVE-2021-0677,CWE-190,"In ccu driver, there is a possible out of boun...",CWE-190->CWE-125,out of bounds read due to an integer overflow
6,CVE-2021-0678,CWE-20,"In apusys, there is a possible out of bounds w...",CWE-20->CWE-787,out of bounds write due to a missing bounds check
8,CVE-2021-0679,CWE-787,"In apusys, there is a possible memory corrupti...",,possible memory corruption
...,...,...,...,...,...
9707,CVE-2022-29897,CWE-20,On various RAD-ISM-900-EN-* devices by PHOENIX...,,due to an improper input validation
9708,CVE-2022-29922,CWE-20,Improper Input Validation vulnerability in the...,,Improper Input Validation
9709,CVE-2022-3001,CWE-20,This vulnerability exists in Milesight Video M...,,improper input handling
9710,CVE-2022-30232,CWE-20,A CWE-20: Improper Input Validation vulnerabil...,,CWE-20: Improper Input Validation


In [54]:
df.Chains.value_counts()

Chains
                                      5771
CWE-1287->CWE-617                       74
CWE-20->CWE-787                         74
CWE-362->CWE-416                        43
CWE-416->CWE-122                        31
                                      ... 
CWE-197->CWE-125, CWE-197->CWE-787       1
CWE-190->CWE-120                         1
CWE-20->CWE-476, CWE-129->CWE-119        1
CWE-1389->CWE-918                        1
CWE-20->CWE-35                           1
Name: count, Length: 437, dtype: int64

In [55]:
df.CVE.value_counts()

CVE
CVE-2022-30233    1
CVE-2022-29198    1
CVE-2022-29197    1
CVE-2022-29196    1
CVE-2022-29195    1
                 ..
CVE-2021-0895     1
CVE-2021-0679     1
CVE-2021-0678     1
CVE-2021-0677     1
CVE-2021-0676     1
Name: count, Length: 6945, dtype: int64

In [56]:
df.Chains.value_counts()


Chains
                                      5771
CWE-1287->CWE-617                       74
CWE-20->CWE-787                         74
CWE-362->CWE-416                        43
CWE-416->CWE-122                        31
                                      ... 
CWE-197->CWE-125, CWE-197->CWE-787       1
CWE-190->CWE-120                         1
CWE-20->CWE-476, CWE-129->CWE-119        1
CWE-1389->CWE-918                        1
CWE-20->CWE-35                           1
Name: count, Length: 437, dtype: int64

In [57]:
df.Weakness_Description.isnull().value_counts()

Weakness_Description
False    6945
Name: count, dtype: int64